In [8]:
import json
from pprint import pprint
Settings = json.load(open('settings.txt'))
pprint(Settings)
import sys
sys.path.insert(0,'../')
from numpy import genfromtxt
import numpy as np
from pak.evaluation import MOTA
from cabbage.data.video import VideoData
from experiments import MOT16_Experiments

root = Settings['data_root']

mot16 = MOT16_Experiments(root)
Hy = VideoData(mot16.get_MOT16_11_trajectories())  # true detections
Hy = Hy.get_n_first_frames(100)
print(Hy.shape)


result = genfromtxt('../build/RESULT.txt', delimiter=' ', dtype='uint32')
print(result[:,2].shape)


# def extract_ids(result):
#     #TODO: make this stuff better..
    
#     candidates = []
#     n = np.max(result[:,1])  # total number of nodes
    
#     Nodes = [i for i in range(n+1)]
    
    
#     for i, j, edge in result:
#         assert(i < j)
#         has_edge = edge == 0
        
#         if has_edge:
#             repres = Nodes[i]
#             Nodes[j] = repres

#     #print(Nodes)     
#     return Nodes

# Nodes = extract_ids(result)

# # translate to sensible id's
# n_unique = len(np.unique(Nodes))
# lookup = {}
# for new,old in zip(list(range(n_unique)), np.unique(Nodes)):
#     lookup[old] = new

# for i in range(len(Nodes)):
#     Nodes[i] = lookup[Nodes[i]]

    
# print("Unique ids:", n_unique)

{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static',
 'graph_solver': '/home/cabbage/build/sample'}
(501, 6)
(7119651,)


In [9]:
print(np.max(result[:,2]))

0
